In [1]:
import pandas as pd
import os
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import f_classif,SelectKBest
from sklearn.pipeline import Pipeline

In [3]:
def load_data(path):
    data=pd.read_csv(path)
    return data

def save_data(data,path):
    dirname="TEST"
    dirpath=os.path.join(path,dirname)
    os.makedirs(dirpath)
    data.to_csv(os.path.join(dirpath,"TRAIN.csv"),index=False)

In [12]:
data=load_data(r"C:\Users\Avijit\Desktop\TEST\data\healthcare_dataset.csv")

In [13]:
data.columns.tolist()

['Name',
 'Age',
 'Gender',
 'Blood Type',
 'Medical Condition',
 'Date of Admission',
 'Doctor',
 'Hospital',
 'Insurance Provider',
 'Billing Amount',
 'Room Number',
 'Admission Type',
 'Discharge Date',
 'Medication',
 'Test Results']

In [14]:
data=data.drop(columns=['Name',
 'Date of Admission',
 'Hospital',
 'Insurance Provider',
 'Admission Type',
 'Discharge Date'])

In [15]:
data.to_csv(r"C:\Users\Avijit\Desktop\TEST\data\healthcare_dataset.csv",index=False)

In [75]:
def processing_data(data):
    numcol=data.select_dtypes(include=['number']).columns.tolist()
    catcol=data.select_dtypes(include=['object']).columns.tolist()
    catpipe=Pipeline(steps=[("LABELENCODING",OrdinalEncoder())])
    numpipe=Pipeline(steps=[("SCALING",StandardScaler())])
    mergepipe=ColumnTransformer(transformers=[("CATEGORICAL_DATA",catpipe,catcol),("NUMERICAL_DATA",numpipe,numcol)],remainder="passthrough")
    finalpipe=Pipeline(steps=[('DATA_PROCESSING',mergepipe)])
    df=finalpipe.fit_transform(data)
    df1=pd.DataFrame(df,columns=catcol+numcol)
    temp=df1.pop("Test Results")
    df1.insert(len(df1.columns.tolist()),"Test Results",temp)
    return df1

In [76]:
df=processing_data(data)

In [78]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [83]:
fc=SelectKBest(score_func=f_classif,k=4)

In [84]:
f1=fc.fit_transform(x,y)

d:\HEALTHCARE\venv\lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [86]:
dff=pd.DataFrame(f1,columns=x.columns[fc.get_support()])

In [88]:
dff=pd.concat([dff,y],axis=1)

In [89]:
dff

,Gender,Medication,Age,Billing Amount,Test Results
0,1.0,3.0,-1.098824,-0.470261,2.0
1,1.0,1.0,0.533639,0.570250,1.0
2,0.0,0.0,1.247842,0.169990,2.0
3,0.0,1.0,-1.200853,0.870465,0.0
4,0.0,4.0,-0.435636,-0.795211,0.0
...,...,...,...,...,...
55495,0.0,4.0,-0.486651,-1.610589,0.0
55496,0.0,0.0,0.482625,0.416462,2.0
55497,0.0,1.0,-0.690708,0.146464,0.0
55498,1.0,1.0,-0.435636,0.486357,0.0


In [90]:
from xgboost import XGBClassifier

xg=XGBClassifier()